In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate

In [3]:
bs = 64

In [9]:
images_path = 'soccer_vs_footy_images'
image_filenames = get_image_files(images_path)
image_filenames[:4]

[PosixPath('soccer_vs_footy_images/soccer_45.jpg'),
 PosixPath('soccer_vs_footy_images/footy_25.JPG'),
 PosixPath('soccer_vs_footy_images/soccer_20.jpg'),
 PosixPath('soccer_vs_footy_images/footy_33.jpg')]

In [20]:
np.random.seed(2)
pat = r'/([^/]+)_\d+.(jpg|jpeg|JPG|JPEG|png)$'

In [13]:
doc(ImageDataBunch.normalize)

In [21]:
data = ImageDataBunch.from_name_re(
    images_path, image_filenames, pat, ds_tfms=get_transforms(), size=224, bs=bs
).normalize()

In [22]:
data.show_batch(rows=3, figsize=(8,7))

OSError: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 647, in __getitem__
    if self.item is None: x,y = self.x[idxs],self.y[idxs]
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 118, in __getitem__
    if isinstance(idxs, Integral): return self.get(idxs)
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/vision/data.py", line 270, in get
    res = self.open(fn)
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/vision/data.py", line 266, in open
    return open_image(fn, convert_mode=self.convert_mode, after_open=self.after_open)
  